In [1]:
!pip install pandas
!pip install sklearn

In [2]:
import pandas as pd
info = pd.read_csv("dataset/WiDS Datathon 2020 Dictionary.csv")

# Dropping Unimportant Columns

In [3]:
drop_cols = ['encounter_id','patient_id','icu_id', 'hospital_id', 'readmission_status']
drop_cols_test = ['encounter_id','patient_id','icu_id','hospital_death','hospital_id', 'readmission_status']

train = pd.read_csv("dataset/training_v2.csv").drop(drop_cols, axis=1)
test = pd.read_csv("dataset/unlabeled.csv").drop(drop_cols_test, axis=1)

In [4]:
target = "hospital_death"

In [5]:
print(f'Rows in train data : {train.shape[0]} and columns in train data: {train.shape[1]}')
print(f'Rows in test data  : {test.shape[0]} and columns in train data: {test.shape[1]}')

Rows in train data : 91713 and columns in train data: 181
Rows in test data  : 39308 and columns in train data: 180


# Data Cleaning - Fill in NA/empty data

In [6]:
train['age'].fillna(train['age'].mean(), inplace = True)
train['bmi'].fillna(train['bmi'].median(), inplace = True)
train['ethnicity'].fillna(train['ethnicity'].mode()[0], inplace = True)
train['weight'].fillna(train['weight'].median(), inplace = True)
train['height'].fillna(train['height'].median(), inplace = True)
train['gender'].fillna(train['gender'].mode()[0], inplace = True)

In [7]:
for i in range(41,169,1):
    train.iloc[:,i] = train.iloc[:,i].fillna(0)

In [8]:
cols2 = train.isna().sum().reset_index()["index"]
val2 = train.isna().sum().reset_index()[0]

In [9]:
for c, v in zip(cols2, val2):
    print(c, v)

hospital_death 0
age 0
bmi 0
elective_surgery 0
ethnicity 0
gender 0
height 0
hospital_admit_source 21409
icu_admit_source 112
icu_stay_type 0
icu_type 0
pre_icu_los_days 0
weight 0
albumin_apache 54379
apache_2_diagnosis 1662
apache_3j_diagnosis 1101
apache_post_operative 0
arf_apache 715
bilirubin_apache 58134
bun_apache 19262
creatinine_apache 18853
fio2_apache 70868
gcs_eyes_apache 1901
gcs_motor_apache 1901
gcs_unable_apache 1037
gcs_verbal_apache 1901
glucose_apache 11036
heart_rate_apache 878
hematocrit_apache 19878
intubated_apache 715
map_apache 994
paco2_apache 70868
paco2_for_ph_apache 70868
pao2_apache 70868
ph_apache 70868
resprate_apache 1234
sodium_apache 18600
temp_apache 4108
urineoutput_apache 48998
ventilated_apache 715
wbc_apache 22012
d1_diasbp_invasive_max 0
d1_diasbp_invasive_min 0
d1_diasbp_max 0
d1_diasbp_min 0
d1_diasbp_noninvasive_max 0
d1_diasbp_noninvasive_min 0
d1_heartrate_max 0
d1_heartrate_min 0
d1_mbp_invasive_max 0
d1_mbp_invasive_min 0
d1_mbp_max 0
d

In [10]:
test['age'].fillna(test['age'].mean(), inplace = True)
test['bmi'].fillna(test['bmi'].median(), inplace = True)
test['ethnicity'].fillna(test['ethnicity'].mode()[0], inplace = True)
test['weight'].fillna(test['weight'].median(), inplace = True)
test['height'].fillna(test['height'].median(), inplace = True)
test['gender'].fillna(test['gender'].mode()[0], inplace = True)

In [11]:
for i in range(41,169,1):
    test.iloc[:,i] = test.iloc[:,i].fillna(0)

In [12]:
cat_cols = ['gcs_verbal_apache',
     'icu_admit_source',
     'icu_type',
     'icu_stay_type',
     'ethnicity',
     'gender',
     'apache_2_bodysystem',
     'gcs_eyes_apache',
     'gcs_motor_apache',
     'apache_3j_bodysystem']

In [13]:
for col in cat_cols:
    train[col] = pd.get_dummies(train[[col]])
    test[col] = pd.get_dummies(test[[col]])

# Training Model

In [14]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier

In [15]:
train = train.dropna()
y = train[target]
X = train.drop(['hospital_admit_source'], axis=1)
X = X.drop(target, axis=1)

X_train, valid_features, Y_train, valid_y = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [16]:
gradient_boost = GradientBoostingClassifier(random_state=1)
random_forest = RandomForestClassifier(n_estimators=100)

voting = VotingClassifier(estimators=[('rfc', random_forest),('gbc', gradient_boost)], voting='soft')
voting = voting.fit(X_train, Y_train)
predict_y = voting.predict(valid_features)

In [48]:
# test = test.dropna()
# test['leukemia'].fillna(test['leukemia'].mode()[0], inplace = True)
# test['lymphoma'].fillna(test['lymphoma'].mode()[0], inplace = True)
# test['albumin_apache'].fillna(test['lymphoma'].mode()[0], inplace = True)
# test['apache_2_diagnosis'].fillna(test['lymphoma'].mode()[0], inplace = True)
# test['apache_3j_diagnosis'].fillna(test['lymphoma'].mode()[0], inplace = True)
# test['arf_apache'].fillna(test['lymphoma'].mode()[0], inplace = True)
# test['bilirubin_apache'].fillna(test['lymphoma'].mode()[0], inplace = True)
# test['solid_tumor_with_metastasis'].fillna(test['solid_tumor_with_metastasis'].mode()[0], inplace = True)

for col in test.columns:
    if test[col].isnull().any().sum():
        test[col].fillna(test[col].mode()[0], inplace=True)

# # test = test.drop(['hospital_admit_source'], axis=1)

# test_copy = test.copy()
# test_copy["hospital_death"] = voting.predict(test)

# test_copy[["encounter_id","hospital_death"]].to_csv("submission1.csv",index=False)
# test_copy[["encounter_id","hospital_death"]].head()

In [ ]:
test_copy['hospital_death'].to_csv("submission1.csv", index=True, index_label='encounter_id')

In [49]:
test.columns[test.isnull().any()]

Index([], dtype='object')